In [ ]:
import pandas as pd

base_path = "../data/"  

reviews = pd.read_csv(base_path + "olist_order_reviews.csv")
orders = pd.read_csv(base_path + "olist_orders.csv")
items = pd.read_csv(base_path + "olist_order_items.csv")
customers = pd.read_csv(base_path + "olist_customers.csv")

# print("orders:", orders.shape)
# print("items:", items.shape)
# print("customers:", customers.shape)

# orders.head()

In [ ]:
orders_reviews = pd.merge(orders, reviews, on="order_id", how="inner")


In [ ]:
reviews.head()

In [ ]:
avg_review_scores = orders_reviews.groupby("customer_id").agg(
    avg_review_score=("review_score", "mean")
).reset_index()


In [ ]:
customer_summary = pd.merge(customer_orders, avg_review_scores, on="customer_id", how="left")
customer_summary.head()


In [ ]:
# import sys
# !{sys.executable} -m pip install textblob


In [ ]:
# import nltk
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('wordnet')

In [ ]:
# pip install deep-translator


In [ ]:
from textblob import TextBlob
from deep_translator import GoogleTranslator

def get_sentiment_v2(text):
    if pd.isnull(text) or text.strip() == "":
        return None
    try:
        translated = GoogleTranslator(source='auto', target='en').translate(text)
        return TextBlob(translated).sentiment.polarity
    except:
        return None


In [ ]:
from tqdm.auto import tqdm
tqdm.pandas()

sample_reviews = reviews.sample(100, random_state=42).copy()
sample_reviews["sentiment_score"] = sample_reviews["review_comment_message"].progress_apply(get_sentiment_v2)


In [ ]:
sample_reviews["review_comment_message"].isnull().sum()

In [ ]:
from deep_translator import GoogleTranslator

def try_translate(text):
    if pd.isnull(text) or text.strip() == "":
        return None
    try:
        return GoogleTranslator(source='auto', target='en').translate(text)
    except:
        return None

# 원문과 번역 컬럼 추가
sample_reviews["translated_comment"] = sample_reviews["review_comment_message"].apply(try_translate)

# 앞 100개만 보기
sample_reviews[["review_comment_message", "translated_comment"]].head(100)


In [ ]:
total_by_score = reviews.groupby('review_score').size()

In [ ]:
text_by_score = reviews[reviews['review_comment_message'].notnull()].groupby('review_score').size()

In [ ]:
text_ratio = (text_by_score / total_by_score).fillna(0).round(2)

In [ ]:
comparison_df = pd.DataFrame({
    'Total Reviews': total_by_score,
    'Reviews with Text': text_by_score,
    'Text Review Ratio': text_ratio
})

print(comparison_df)

In [ ]:
plt.figure(figsize=(8, 5))
text_ratio.sort_index().plot(kind='bar', color='skyblue')

plt.title('Text Review Ratio by Star Rating')
plt.xlabel('Review Score')
plt.ylabel('Ratio of Reviews with Text')
plt.ylim(0, 1)
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()